In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.display import display
from pulp import LpMaximize, LpProblem, LpStatus, LpVariable, lpSum

In [6]:
# Import data from Excel files
initial_inventory = pd.read_csv('data/Prosacco-Initial-Inventory.csv')
order_report = pd.read_csv('data/Prosacco-order-report.csv')
production_plan = pd.read_excel('data/Prosacco-production-plan.xlsx')

In [7]:
# Display the heads of each dataframe
print('Initial Inventory:')
print(initial_inventory.head())

print('Order Report:')
print(order_report.head())

print('Production Plan:')
print(production_plan.head())

Initial Inventory:
           Category     SKU Warehouse  Available
0    Fresh Packaged  FP2020      Kern       4063
1    Fresh Packaged  FP3055      Kern       2032
2  Healthy Beverage  HB0156      Kern        148
3  Healthy Beverage  HB1016      Kern       4974
4           Healthy  HT1045      Kern        121
Order Report:
         Country           City  PO/ORDER#        Categories     SKU  QTY ORD  \
0  United States  NEW YORK CITY     102981    Fresh Packaged  FP2020       52   
1  United States  NEW YORK CITY     102980    Organic Beauty  OY2545       52   
2  United States    Los Angeles     102979    Organic Beauty  OY2545        3   
3  United States    Los Angeles     102978  Healthy Beverage  HB1016        3   
4  United States    Los Angeles     102977           Healthy  HT1064       90   

  CHANNEL WAREHOUSE Pick-Up / Delivery   Customer  SALES $  EXPECTED  
0  Retail      Kern           Delivery  Customer1     6500     45236  
1  Retail      Kern           Delivery  Cust

In [8]:
# Display info and missing values for each dataframe

print('Initial Inventory Info:')
print(initial_inventory.info())
print('\nMissing values:')
print(initial_inventory.isnull().sum())

print('\nOrder Report Info:')
print(order_report.info())
print('\nMissing values:')
print(order_report.isnull().sum())

print('\nProduction Plan Info:')
print(production_plan.info())
print('\nMissing values:')
print(production_plan.isnull().sum())

Initial Inventory Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Category   12 non-null     object
 1   SKU        12 non-null     object
 2   Warehouse  12 non-null     object
 3   Available  12 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 512.0+ bytes
None

Missing values:
Category     0
SKU          0
Warehouse    0
Available    0
dtype: int64

Order Report Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country             2880 non-null   object
 1   City                2880 non-null   object
 2   PO/ORDER#           2880 non-null   int64 
 3   Categories          2880 non-null   object
 4   SKU                 2880 non-null   object
 5   QTY ORD             28

In [9]:
# Clean the data: drop rows with missing values and reset index for all dataframes
initial_inventory_clean = initial_inventory.dropna().reset_index(drop=True)
order_report_clean = order_report.dropna().reset_index(drop=True)
production_plan_clean = production_plan.dropna().reset_index(drop=True)

print('Initial Inventory after cleaning:')
print(initial_inventory_clean.info())
print('\nOrder Report after cleaning:')
print(order_report_clean.info())
print('\nProduction Plan after cleaning:')
print(production_plan_clean.info())

Initial Inventory after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Category   12 non-null     object
 1   SKU        12 non-null     object
 2   Warehouse  12 non-null     object
 3   Available  12 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 512.0+ bytes
None

Order Report after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country             2880 non-null   object
 1   City                2880 non-null   object
 2   PO/ORDER#           2880 non-null   int64 
 3   Categories          2880 non-null   object
 4   SKU                 2880 non-null   object
 5   QTY ORD             2880 non-null   int64 
 6   CHANNEL             2880 non-null   object
 

In [13]:
# Inventory Analysis

# List all SKUs and their inventory counts
print('All SKUs and Inventory Counts:')
print(initial_inventory_clean[['SKU', 'Available']])

# Identify SKUs with zero or low inventory (threshold: 10 units)
low_inventory = initial_inventory_clean[initial_inventory_clean['Available'] <= 10]
print('\nSKUs with Zero or Low Inventory (<=10 units):')
print(low_inventory[['SKU', 'Available']])

# Find the SKU with the highest inventory
max_inventory_row = initial_inventory_clean.loc[initial_inventory_clean['Available'].idxmax()]
print(f"\nSKU with Highest Inventory: {max_inventory_row['SKU']} ({max_inventory_row['Available']} units)")

# Focus on key SKU FP2020
fp2020_row = initial_inventory_clean[initial_inventory_clean['SKU'] == 'FP2020']
if not fp2020_row.empty:
    print(f"\nFP2020 Inventory Status: {fp2020_row.iloc[0]['Available']} units")
else:
    print('\nFP2020 not found in inventory.')

All SKUs and Inventory Counts:
       SKU  Available
0   FP2020       4063
1   FP3055       2032
2   HB0156        148
3   HB1016       4974
4   HT1045        121
5   HT1064        138
6   HT2054        187
7   OB1265        130
8   OF1060         32
9   OF2035        185
10  OP8025       2073
11  OY2545       4250

SKUs with Zero or Low Inventory (<=10 units):
Empty DataFrame
Columns: [SKU, Available]
Index: []

SKU with Highest Inventory: HB1016 (4974 units)

FP2020 Inventory Status: 4063 units


In [14]:
# Order Analysis

# Summarize total orders by SKU
orders_by_sku = order_report_clean.groupby('SKU').size().reset_index(name='Order Count')
print('Total Orders by SKU:')
print(orders_by_sku)

# Identify SKUs with high order volumes (top 3)
top_orders = orders_by_sku.sort_values('Order Count', ascending=False).head(3)
print('\nTop 3 SKUs by Order Volume:')
print(top_orders)

# Identify SKUs with low order volumes (bottom 3)
low_orders = orders_by_sku.sort_values('Order Count', ascending=True).head(3)
print('\nBottom 3 SKUs by Order Volume:')
print(low_orders)

# Compare order volumes to inventory levels
comparison = pd.merge(orders_by_sku, initial_inventory_clean[['SKU', 'Available']], on='SKU', how='left')
print('\nOrder Volume vs Inventory Level:')
print(comparison)

Total Orders by SKU:
       SKU  Order Count
0   FP2020          625
1   FP3055          253
2   HB0156           52
3   HB1016          747
4   HT1045           46
5   HT1064           13
6   HT2054           13
7   OB1265           47
8   OF1060           10
9   OF2035           56
10  OP8025          346
11  OY2545          672

Top 3 SKUs by Order Volume:
       SKU  Order Count
3   HB1016          747
11  OY2545          672
0   FP2020          625

Bottom 3 SKUs by Order Volume:
      SKU  Order Count
8  OF1060           10
6  HT2054           13
5  HT1064           13

Order Volume vs Inventory Level:
       SKU  Order Count  Available
0   FP2020          625       4063
1   FP3055          253       2032
2   HB0156           52        148
3   HB1016          747       4974
4   HT1045           46        121
5   HT1064           13        138
6   HT2054           13        187
7   OB1265           47        130
8   OF1060           10         32
9   OF2035           56        185

In [15]:
# Production Plan Analysis
# Visualize production levels by SKU and week
if 'Week' in production_plan_clean.columns and 'SKU' in production_plan_clean.columns and 'Produced' in production_plan_clean.columns:
    pivot = production_plan_clean.pivot_table(index='Week', columns='SKU', values='Produced', aggfunc='sum')
    pivot.plot(figsize=(12,6))
    plt.title('Production Levels by SKU and Week')
    plt.ylabel('Produced Units')
    plt.xlabel('Week')
    plt.legend(title='SKU')
    plt.show()

    # Identify weeks with zero production for any SKU
    zero_prod = production_plan_clean[production_plan_clean['Produced'] == 0]
    print('\nWeeks with Zero Production for Any SKU:')
    print(zero_prod[['Week', 'SKU']])

    # Highlight delays or gaps in production for FP2020
    fp2020_zero = production_plan_clean[(production_plan_clean['SKU'] == 'FP2020') & (production_plan_clean['Produced'] == 0)]
    print('\nFP2020 Zero Production Weeks:')
    print(fp2020_zero['Week'].unique())
else:
    print('Required columns (Week, SKU, Produced) not found in production_plan_clean.')

Required columns (Week, SKU, Produced) not found in production_plan_clean.


In [16]:
# Display column names to identify correct columns for production analysis
print('Production Plan Clean Columns:')
print(production_plan_clean.columns)

Production Plan Clean Columns:
Index(['Production Plan', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')


In [17]:
# Display the first few rows to understand the structure of production_plan_clean
print(production_plan_clean.head())

Empty DataFrame
Columns: [Production Plan, Unnamed: 1, Unnamed: 2, Unnamed: 3, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, Unnamed: 8, Unnamed: 9, Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 13, Unnamed: 14, Unnamed: 15]
Index: []


In [21]:
# Re-import the production plan data without dropping missing values and inspect the first few rows
production_plan_matrix = pd.read_excel('data/Prosacco-production-plan.xlsx', header=1)
production_plan_matrix = production_plan_matrix[production_plan_matrix['SKU'].notnull()]
production_plan_long = production_plan_matrix.melt(id_vars=['SKU'], var_name='Week', value_name='Produced')
print(production_plan_long.head(10))

KeyError: 'SKU'

In [22]:
# Check column names after reading the Excel file
production_plan_matrix = pd.read_excel('data/Prosacco-production-plan.xlsx', header=1)
print('Columns:', production_plan_matrix.columns.tolist())

Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']


In [23]:
# Clean and reshape the production plan data for analysis

# Use the second row as header (row index 1) and skip the first row
production_plan_matrix = pd.read_excel('data/Prosacco-production-plan.xlsx', header=1)

# Remove any rows where SKU is NaN or not a string
production_plan_matrix = production_plan_matrix[production_plan_matrix['SKU'].notnull()]

# Melt the dataframe to long format: SKU, Week, Produced
production_plan_long = production_plan_matrix.melt(id_vars=['SKU'], var_name='Week', value_name='Produced')

# Display the first few rows of the reshaped data
print(production_plan_long.head(10))

KeyError: 'SKU'

In [20]:
# Production Plan Analysis: Visualize and detect gaps/delays
import matplotlib.pyplot as plt

# Pivot for visualization: Weeks as index, SKUs as columns
pivot = production_plan_long.pivot_table(index='Week', columns='SKU', values='Produced', aggfunc='sum')
pivot.plot(figsize=(12,6))
plt.title('Production Levels by SKU and Week')
plt.ylabel('Produced Units')
plt.xlabel('Week')
plt.legend(title='SKU')
plt.show()

# Identify weeks with zero production for any SKU
zero_prod = production_plan_long[production_plan_long['Produced'] == 0]
print('\nWeeks with Zero Production for Any SKU:')
print(zero_prod[['Week', 'SKU']])

# Highlight delays or gaps in production for FP2020
fp2020_zero = production_plan_long[(production_plan_long['SKU'] == 'FP2020') & (production_plan_long['Produced'] == 0)]
print('\nFP2020 Zero Production Weeks:')
print(fp2020_zero['Week'].unique())

NameError: name 'production_plan_long' is not defined

## 6. Linear programming: optimised allocation
We now construct a linear program that reallocates Prosacco supply across cities to maximise fulfilled demand. The model treats each SKU separately, combining on-hand inventory with the forward production plan to determine weekly supply. Decision variables capture the quantity shipped from the warehouse to each market, constrained by supply availability and customer demand.

In [ ]:
# Prepare aggregated demand and supply for optimisation
demand_by_city_sku = (
    order_report_clean.groupby(["City", "SKU"])["QTY ORD"].sum().reset_index(name="Demand")
)
demand_by_city_sku = demand_by_city_sku[demand_by_city_sku["Demand"] > 0]

inventory_supply = initial_inventory_clean.groupby("SKU")["Available"].sum().rename("Inventory")
production_supply = production_plan_long.groupby("SKU")["Produced"].sum().rename("Production") if "Produced" in production_plan_long.columns else pd.Series(dtype=float)

supply_by_sku = pd.concat([inventory_supply, production_supply], axis=1).fillna(0)
supply_by_sku["TotalSupply"] = supply_by_sku.sum(axis=1)

skus = sorted(set(demand_by_city_sku["SKU"]) | set(supply_by_sku.index))
cities = sorted(demand_by_city_sku["City"].unique())

print("SKUs considered:", skus)
print("Cities considered:", cities)

In [ ]:
# Load transportation lane configuration (costs & capacities)
lane_config = pd.read_csv("data/Prosacco-lane-costs.csv")
lane_config["City"] = lane_config["City"].str.strip()
lane_config.set_index("City", inplace=True)

lane_capacity = lane_config["MaxCapacity"]
lane_cost = lane_config["CostPerUnit"]
lane_cost_lookup = lane_cost.to_dict()
lane_capacity_lookup = lane_capacity.to_dict()
avg_cost = lane_cost.mean()
missing_cities = sorted(set(cities) - set(lane_config.index))
if missing_cities:
    print("Warning: missing lane data for", missing_cities)
else:
    print("All cities covered in lane configuration.")

cost_weight = 0.0015  # penalty weight translating cost into service-equivalent units
print(f"Average lane cost: {avg_cost:.2f} | Cost weight: {cost_weight}")

In [ ]:
# Formulate and solve the linear program with cost & capacity considerations
shipments = {
    (row["SKU"], row["City"]): LpVariable(
        f"ship_{row['SKU']}_{row['City'].replace(' ', '_')}", lowBound=0
    )
    for _, row in demand_by_city_sku.iterrows()
}

cost_per_lane = {
    (row["SKU"], row["City"]): lane_cost_lookup.get(row["City"], avg_cost)
    for _, row in demand_by_city_sku.iterrows()
}

model = LpProblem("Prosacco_Service_vs_Cost", LpMaximize)
service_expr = lpSum(var for var in shipments.values())
cost_expr = lpSum(cost_per_lane[key] * var for key, var in shipments.items())
model += service_expr - cost_weight * cost_expr, "Maximise_Service_Minimise_Cost"

for sku in skus:
    available = float(supply_by_sku.loc[sku, "TotalSupply"]) if sku in supply_by_sku.index else 0.0
    relevant_vars = [var for (s, _), var in shipments.items() if s == sku]
    if relevant_vars:
        model += lpSum(relevant_vars) <= available, f"supply_{sku}"

for city in cities:
    cap = lane_capacity_lookup.get(city)
    lane_vars = [var for (sku, c), var in shipments.items() if c == city]
    if lane_vars and cap is not None:
        model += lpSum(lane_vars) <= cap, f"lane_capacity_{city.replace(' ', '_')}"

for (sku, city), var in shipments.items():
    demand = float(demand_by_city_sku.loc[(demand_by_city_sku["SKU"] == sku) & (demand_by_city_sku["City"] == city), "Demand"].sum())
    model += var <= demand, f"demand_{sku}_{city}"

status = model.solve()
total_service = service_expr.value()
total_cost = cost_expr.value()
print("Solver status:", LpStatus[status])
print("Objective value (service - weighted cost):", model.objective.value())
print(f"Fulfilled units: {total_service:.0f} | Estimated transport cost: ${total_cost:,.2f}")

In [ ]:
# Summarise the optimisation outcome
allocation_df = pd.DataFrame([
    {"SKU": sku, "City": city, "Allocated": var.value() or 0.0}
    for (sku, city), var in shipments.items()
])
allocation_df["CostPerUnit"] = allocation_df["City"].map(lane_cost_lookup).fillna(avg_cost)
allocation_df["TransportCost"] = allocation_df["Allocated"] * allocation_df["CostPerUnit"]

sku_demand = demand_by_city_sku.groupby("SKU")["Demand"].sum()
sku_fulfilled = allocation_df.groupby("SKU")["Allocated"].sum()
sku_results = pd.concat([sku_demand, sku_fulfilled], axis=1).fillna(0)
sku_results.columns = ["Demand", "Allocated"]
sku_results["FillRate"] = sku_results["Allocated"] / sku_results["Demand"].replace({0: np.nan})
sku_results["RemainingSupply"] = supply_by_sku["TotalSupply"].reindex(sku_results.index).fillna(0) - sku_results["Allocated"]
sku_results["TransportCost"] = allocation_df.groupby("SKU")["TransportCost"].sum().reindex(sku_results.index).fillna(0)
sku_results["Shortage"] = sku_results["Demand"] - sku_results["Allocated"]
print("SKU-level summary:")
display(sku_results.round(2))

city_fulfilled = allocation_df.groupby("City")["Allocated"].sum()
city_demand = demand_by_city_sku.groupby("City")["Demand"].sum()
city_cost = allocation_df.groupby("City")["TransportCost"].sum()
city_results = pd.concat([city_demand, city_fulfilled, city_cost], axis=1).fillna(0)
city_results.columns = ["Demand", "Allocated", "TransportCost"]
city_results["FillRate"] = city_results["Allocated"] / city_results["Demand"].replace({0: np.nan})

lane_capacity_used = allocation_df.groupby("City")["Allocated"].sum().reindex(lane_capacity.index).fillna(0)
lane_summary = pd.concat(
    [lane_capacity.rename("Capacity"), lane_capacity_used.rename("Utilised")], axis=1
).fillna(0)
lane_summary["UtilisationPct"] = np.where(
    lane_summary["Capacity"] > 0,
    (lane_summary["Utilised"] / lane_summary["Capacity"]) * 100,
    np.nan
)

print("\nCity-level summary:")
display(city_results.round(2))

print("\nLane capacity utilisation:")
display(lane_summary.round({"Capacity": 0, "Utilised": 0, "UtilisationPct": 1}))

total_cost_reported = allocation_df["TransportCost"].sum()
print(f"\nTotal transport cost (from allocation): ${total_cost_reported:,.2f}")
print("\nAllocation preview:")
display(allocation_df.sort_values(["SKU", "City"]).head(10))

### Interpretation
- **Objective**: maximise total units served while penalising expensive transport moves (weight = 0.0015).
- **Fill rate lift**: compare the optimisation fill rates to the descriptive results above to identify SKUs where targeted reallocations deliver the biggest gains.
- **Remaining supply**: positive balances indicate latent capacity that could be reassigned to future demand or held as safety stock.
- **Transport spend**: the total cost output quantifies the trade-off between service and spend—tune the weight or lane tariffs to assess sensitivities.
- **Lane utilisation**: utilisation percentages highlight routes that are capacity constrained and may require additional carriers or alternative fulfillment paths.

# Analysis Plan for Sales Coverage Optimization

1. **Data Overview**
   - Display heads and info for Initial Inventory, Order Report, and Production Plan.
   - Check for missing values and data types.

2. **Inventory Analysis**
   - List all SKUs and their inventory counts.
   - Identify SKUs with zero or low inventory.
   - Find the SKU with the highest inventory.
   - Focus on key SKUs (e.g., FP2020) and their inventory status.

3. **Order Analysis**
   - Summarize total orders by SKU.
   - Identify SKUs with high/low order volumes.
   - Compare order volumes to inventory levels.

4. **Production Plan Analysis**
   - Visualize production levels by SKU and week.
   - Identify weeks with zero production for any SKU.
   - Highlight delays or gaps in production (e.g., FP2020 in weeks 39–41).

5. **Supply-Demand Alignment**
   - Compare inventory, orders, and production for each SKU.
   - Flag SKUs at risk of stockouts or overstock.

6. **Recommendations**
   - Suggest actions for SKUs with mismatched supply and demand.
   - Recommend production adjustments or inventory reallocation.